In [ ]:
groups_to_keep =[r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",# hash-tags
                r"""
    (?:
      [<>]?
      [:;=]                     # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth      
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=]                     # eyes
      [<>]?
    )""" #smileys
                ]

regex_to_keep=  r'('+'|'.join(groups_to_keep)+')'

In [ ]:
mapping_motifs = pandas.DataFrame(np.transpose([motifs,motifs]),columns=['micro_src','micro_dest'])
seuil_confiance = 90

def fuzzy_logic(motif):
    resultat = process.extractOne(motif, themes_edf["Désignation"])
    if resultat[1] > seuil_confiance :
        return resultat[0]
    else : 
        return np.nan

def get_macro_category(micro):
    if not pandas.isnull(micro) :
        return themes_edf["Thème/Motif"][themes_edf["Désignation"]== micro].values[0]
    else:
        return 'Autres'
    
mapping_motifs['micro_dest'] = mapping_motifs['micro_dest'].map(fuzzy_logic)
mapping_motifs['macro'] =  mapping_motifs['micro_dest'].copy(deep=True)

mapping_motifs['macro'] = mapping_motifs['macro'].map(get_macro_category)

In [ ]:
def remove_patterns(verbatim) :
    if re.search(regex_to_delete, verbatim) :
        return re.sub(regex_to_delete, "", verbatim) 
    return verbatim

def no_accents(s):
    s= unicodedata.normalize('NFD',s)
    return str(s.encode('ascii','ignore'), 'utf-8')

groups_to_delete = [r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
                    r'(?:(?:\d+,?)+(?:\.?\d+)?)',# numbers
                    r'([_\-]+)' # underscores and dashes
    ] 

                 
regex_to_delete = r'('+'|'.join(groups_to_delete)+')'

word_tokenizer = RegexpTokenizer('\w+', gaps=False)

frenchStemmer= SnowballStemmer('french', ignore_stopwords=True)

stop_words_french = stopwords.words('french')
stop_words_french_normalized =[no_accents(stop) for stop in stop_words_french]

JAR_PATH = 'C:/Users/MACH055/Documents/code/stanford-postagger-full-2015-12-09/stanford-postagger.jar'
French_MODEL_PATH = 'C:/Users/MACH055/Documents/code/stanford-postagger-full-2015-12-09/models/french.tagger'
FrenchPostagger = StanfordPOSTagger(French_MODEL_PATH, JAR_PATH, encoding='utf8')

french_tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

In [ ]:
def preprocess_data(data, cleaning=True, stemming=True, pos_tagging_lemmatization=False):
    clean_verbatims = list()
    pos_tagged = list()
    
    # POS-Tagging
    #if pos_tagging :
     #   sentences = [str(v).split() for v in data]
    #  pos_tagged = FrenchPostagger.tag_sents(sentences)
       
    
    for verbatim in data :
        
        # remove numbers, html ...
        if cleaning :
            verbatim_i = remove_patterns(str(verbatim))
        
        # POS-Tagging
        if pos_tagging_lemmatization :
            temp_pos = french_tagger.tag_text(verbatim_i)
            tags = treetaggerwrapper.make_tags(temp_pos)
            pos_tagged.append(tags)
            
            # Fonction permettant d'obtenir le lemme d'un objet Tag ou recuperer le mot si un objet NotTag
            getlemma_pos = lambda x : operator.attrgetter('lemma')(x) if (
                isinstance(x,treetaggerwrapper.Tag)) else operator.attrgetter('what')(x)
            verbatim_i = ' '.join([getlemma_pos(tag) for tag in tags])
            
        # remove accents 
        tokens = word_tokenizer.tokenize(no_accents(verbatim_i))
            
        # remove stop words + to lower case + stemming
        if stemming :
            tokens_w_s = [frenchStemmer.stem(i.lower()) for i in tokens if (i not in stop_words_french_normalized) & (len(i)>2)]
            
        if pos_tagging_lemmatization :
            tokens_w_s = [i for i in tokens if (i not in stop_words_french_normalized) & (len(i)>2)]
            
        # concat words
        clean_verbatims.append(' '.join(tokens_w_s))
        
    return clean_verbatims, pos_tagged